In [1]:
# !pip install Faker

In [2]:
import pandas as pd
import random

from faker import Faker
fake = Faker()
Faker.seed(4321)

# Users

In [3]:
# n_rows = 10213

# customer_ids = random.sample(range(100000001, 999999999), n_rows)
# mail_domains = [fake.profile()['mail'].split('@')[-1] for _ in range(0, n_rows)]

# list_customers = []
# for _ in range(n_rows):
#     current_profile = fake.profile()
#     current_row = {
#         'user_id': customer_ids.pop(),
#         'first_name': current_profile['name'].split(' ')[0],
#         'last_name': current_profile['name'].split(' ')[1],
#         'is_active': random.choices(population=[True, False], weights=[0.97, 0.03])[0],
#         'status': random.choices(population=[None, 'VIP', 'Platinum'], weights=[0.9, 0.07, 0.03])[0],
#         'sex': current_profile['sex'],
#         'birthdate': fake.date_between(start_date='-67y', end_date='-16y'),
#         'created_date': fake.date_between(start_date='-3y', end_date='today'),
#         'newsletter_consent': random.choices(population=[True, False], weights=[0.27, 0.73])[0],
#         'email': '.'.join(current_profile['name'].lower().split(' ')) + '@' + mail_domains[random.randint(0, n_rows-1)],
#         'address': current_profile['address']
#     }
#     list_customers.append(current_row)
    
# df_customers = pd.DataFrame(list_customers)
# df_customers.to_csv('./output/users.csv', index=False)

# Order details

In [125]:
# n_rows = 145000

# order_id = 1
# position_id = 1
# product_ids = range(87)
# product_ids_prices = {product_id: round(random.uniform(5.0, 20.0), 2) for product_id in product_ids}

# list_orders = []
# for _ in range(n_rows):
#     product_id = random.choice(product_ids)
    
#     current_profile = fake.profile()
#     current_row = {
#         'order_id': order_id,
#         'product_id': product_id,
#         'quantity': random.choices(population=[1, 2, 3], weights=[0.6, 0.3, 0.1])[0] if random.randrange(0, 11) != 1 else random.randrange(4, 30),
#         'price_net': product_ids_prices[product_id],
#         'amount_tax': round(product_ids_prices[product_id] * 0.21, 2),
#         'price_gross': round(product_ids_prices[product_id] * 1.21, 2),
#     }
#     list_orders.append(current_row)
    
#     order_id += random.choices(population=[0, 1], weights=[0.1, 0.9])[0]
    
# df_order_positions = pd.DataFrame(list_orders)
# df_order_positions['position_num'] = df_order_positions.groupby('order_id')['product_id'].rank(method='dense')
# df_order_positions = df_order_positions['order_id', 'position_num', 'product_id', 'quantity', 'price_net', 'amount_tax', 'price_gross']

# df_order_positions.to_csv('./output/order_positions.csv', index=False)

# Orders

In [149]:
df_orders = df_order_positions.copy()

df_orders = (
    df_orders
    .groupby('order_id', as_index=False)
    .agg(
        amount_net=('price_net', 'sum'),
        amount_tax=('amount_tax', 'sum'),
        amount_gross=('price_gross', 'sum'),
    )
)

order_dates = sorted([fake.date_between(start_date='-3y', end_date='today') for tran in range(df_orders['order_id'].nunique())])
df_orders['order_date'] = order_dates

df_orders['payment_delay'] = [
    random.choices(population=[1, 2, 3, 4, 5, 6, 7], weights=[0.05, 0.08, 0.09, 0.13, 0.15, 0.2, 0.3])[0]
    if random.randrange(0, 11) != 1
    else random.randrange(7, 60)
    for tran in range(df_orders['order_id'].nunique())
]

df_orders['payment_date'] = df_orders.apply(lambda row: row['order_date'] + pd.DateOffset(days=row['payment_delay']), axis=1)
df_orders['payment_id'] = df_orders['payment_date'].rank(method='first')

df_orders['seller_id'] = df_orders['order_id'].map(lambda x: df_customers['user_id'].sample().values[0])
df_orders['buyer_id'] = df_orders['order_id'].map(lambda x: df_customers['user_id'].sample().values[0])

In [197]:
df_orders_to_save = df_orders[['order_id', 'order_date', 'payment_id', 'amount_net', 'amount_tax', 'amount_gross', 'buyer_id', 'seller_id']]
df_orders_to_save.to_csv('./output/orders.csv', index=False)

In [196]:
df_orders_to_save.head()

,order_id,order_date,payment_id,amount_net,amount_tax,amount_gross,buyer_id,seller_id
0,1,2019-05-21,3910.0,15.05,3.16,18.21,235883375,516935527
1,2,2019-05-21,5.0,6.18,1.30,7.48,157936606,865627988
2,3,2019-05-21,37.0,12.05,2.53,14.58,907473844,958496623
3,4,2019-05-21,38.0,11.18,2.35,13.53,162762102,745446076
4,5,2019-05-21,83.0,17.73,3.72,21.45,579695060,723421826


In [194]:
df_orders.head()

,order_id,amount_net,amount_tax,amount_gross,order_date,payment_delay,payment_date,payment_id,seller_id,buyer_id
0,1,15.05,3.16,18.21,2019-05-21,40,2019-06-30,3910.0,516935527,235883375
1,2,6.18,1.30,7.48,2019-05-21,2,2019-05-23,5.0,865627988,157936606
2,3,12.05,2.53,14.58,2019-05-21,4,2019-05-25,37.0,958496623,907473844
3,4,11.18,2.35,13.53,2019-05-21,4,2019-05-25,38.0,745446076,162762102
4,5,17.73,3.72,21.45,2019-05-21,5,2019-05-26,83.0,723421826,579695060


In [ ]:
# order_id (string)
# order_date (date)
# payment_id (string)
# amount_net (numeric)
# amount_tax (numeric)
# amount_gross (numeric)
# buyer_id (string)
# seller_id (string)

In [143]:
df_orders.shape

(130593, 4)

In [140]:
df_orders.head()

,order_id,product_id,quantity,price_net,amount_tax,price_gross,position_num
0,1,76,1,15.05,3.16,18.21,1.0
1,2,34,2,6.18,1.30,7.48,1.0
2,3,64,2,12.05,2.53,14.58,1.0
3,4,43,1,11.18,2.35,13.53,1.0
4,5,14,3,17.73,3.72,21.45,1.0


In [145]:
len(order_dates)

130593

In [144]:


#payment_id

In [ ]:
n_rows = 145000

order_ids = random.sample(range(1000000001, 9999999999), n_rows)
payment_ids = random.sample(range(30000001, 39999999), n_rows)

list_orders = []
for _ in range(n_rows):
    current_profile = fake.profile()
    current_row = {
        'order_id': order_ids.pop(),
        'order_date': fake.date_between(start_date='-3y', end_date='today')
        'payment_id': payment_ids.pop()
        'amount_net': 
        'amount_tax': 
        'amount_gross': 
        'buyer_id': 
        'seller_id': 
#         'first_name': current_profile['name'].split(' ')[0],
#         'last_name': current_profile['name'].split(' ')[1],
#         'is_active': random.choices(population=[True, False], weights=[0.97, 0.03])[0],
#         'status': random.choices(population=[None, 'VIP', 'Platinum'], weights=[0.9, 0.07, 0.03])[0],
#         'sex': current_profile['sex'],
#         'birthdate': fake.date_between(start_date='-67y', end_date='-16y'),
#         'created_date': fake.date_between(start_date='-3y', end_date='today'),
#         'newsletter_consent': random.choices(population=[True, False], weights=[0.27, 0.73])[0],
#         'email': '.'.join(current_profile['name'].lower().split(' ')) + '@' + mail_domains[random.randint(0, n_rows-1)],
#         'address': current_profile['address']
    }
    list_orders.append(current_row)
    
df_orders = pd.DataFrame(list_orders)
df_orders.to_csv('./output/orders.csv', index=False)